In [ ]:
# adjust model parameters to start from a steady state
# update reaction parameters informed by Palla2021
# original model from Jahagirdar2019


In [2]:
import pandas as pd


In [6]:
# step 1: get steady state values for initialization

# load steady state samples
temp_df = pd.read_csv('/Users/Finn/Documents/MATLAB/healthyBaseline_reduced_90h.csv',
                      index_col=0)

# get mean values for each metabolite
steady_initial_dict = dict(zip(list(temp_df.columns),
                               temp_df.values.mean(axis=0)))

# example initialization concentration
steady_initial_dict['PC']

0.47214029734652135

In [ ]:
# step 2: open up AA model and poke around
# original model from jahagirdar2019

In [7]:

model_path = '/Users/Finn/Documents/MATLAB/ArachidonicAcid_model.xml'

def load_program(path: str) -> list:
    # load the program
    with open(path,'r') as file:
        prog = file.readlines()
    return prog

prog = load_program(model_path)
print('n lines:',len(prog))
prog[:15]

n lines: 12024


['<?xml version="1.0" encoding="UTF-8"?>\n',
 '<!-- Created by COPASI version 4.22 (Build 170) on 2018-09-24 14:07 with libSBML version 5.15.3. -->\n',
 '<sbml xmlns="http://www.sbml.org/sbml/level2/version4" level="2" version="4">\n',
 '  <model metaid="COPASI0" id="New_Model" name="New Model">\n',
 '    <annotation>\n',
 '      <COPASI xmlns="http://www.copasi.org/static/sbml">\n',
 '        <rdf:RDF xmlns:dcterms="http://purl.org/dc/terms/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">\n',
 '          <rdf:Description rdf:about="#COPASI0">\n',
 '            <dcterms:created>\n',
 '              <rdf:Description>\n',
 '                <dcterms:W3CDTF>2018-03-26T15:39:07Z</dcterms:W3CDTF>\n',
 '              </rdf:Description>\n',
 '            </dcterms:created>\n',
 '          </rdf:Description>\n',
 '        </rdf:RDF>\n']

In [16]:
# check that we can identify each metabolite

# loop through lines, filter for "initialConcentration"
# get metabolite
# lookup steady state concentration from previous run
# replace initial concentration
# boost that value?

met = 0
for idx, _line in enumerate(prog):
    if 'initialConcentration' in _line:
        met += 1
        example_idx = idx
        example_line = _line
print('n metabolites found:', met, '\n')

print('example line:')
print(example_idx, example_line)

n metabolites found: 83 

example line:
7595       <species metaid="COPASI57" id="d12_PGJ2" name="d12_PGJ2" compartment="compartment" initialConcentration="0" boundaryCondition="false" constant="false">



In [17]:
# update the initial concentrations

def update_line(line, concentration_dictionary):
    
    temp_metabolite = line.split('name="')[1].split('"')[0]
    temp_conc = line.split('initialConcentration="')[1]
    temp_steady_value = str(concentration_dictionary[temp_metabolite])
    # recombine strings
    _end_string = '"'.join(temp_conc.split('"')[1:])
    _start_string = line.split('initialConcentration="')[0] + 'initialConcentration="'
    return _start_string + temp_steady_value + '"' + _end_string


In [18]:
# update all the metabolites to steady state values ("healthy" initialization)

prog2 = list()
for idx, _line in enumerate(prog):
    if 'initialConcentration' in _line:
        prog2.append(update_line(line = _line,
                                 concentration_dictionary=steady_initial_dict
                                ))
    else:
        prog2.append(_line)

In [19]:
# save the steady state initial concentrations
def save_to_file(some_list, path):
    with open(path, "w") as file:
        for _line in some_list:
            file.write(_line)
            
save_to_file(some_list = prog2,
             path = '/Users/Finn/desktop/healthy_model_final.xml')

In [ ]:

# specific 15-PGDH enzyme reaction parameters updated in copasi
# 1.1.2.5.3   PGE2 -> 15k_PGE2
# 1.1.2.4.4   PGD2 -> 15k_PGD2
